In [29]:
!pip install optuna
!pip install optuna-integration 'pytorch_lightning'

In [30]:
import os
import random
import numpy as np
import pandas as pd
import optuna
from optuna.integration import PyTorchLightningPruningCallback  # optional, not required

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from datasets import load_from_disk
from PIL import Image

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cpu


In [31]:
RANDOM_SEED = 42
BATCH_SIZE = 32
EPOCHS = 25
TUNE_EPOCHS = 5      # for Bayesian optimisation
IMG_SIZE = 224
PATCH_SIZE = 16
NUM_CLASSES = 2 

TUNE_EPOCHS = 5

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

Right now, your functions use global state (model, optimizer, LAMBDA_ATTR, train_loader, val_loader, etc.).
We can exploit that and just re-assign the globals inside the Optuna objective to avoid refactoring everything.

Idea: for each trial we:
* create a new model with its own drop
* create a new optimizer (lr, weight_decay)
* set LAMBDA_ATTR
* train for TUNE_EPOCHS
* return best validation accuracy seen

In [32]:
TRAIN_VAL_PATH = "processed_bird_data"
TEST_PATH = "processed_bird_test_data"

print("Loading train/val dataset from:", TRAIN_VAL_PATH)
full_ds = load_from_disk(TRAIN_VAL_PATH)
train_hf = full_ds["train"]
val_hf = full_ds["validation"]

print("Train size:", len(train_hf))
print("Val size:", len(val_hf))

print("\nLoading test dataset from:", TEST_PATH)
test_hf = load_from_disk(TEST_PATH)
print("Test size:", len(test_hf))

# attributes
ATTR_PATH = "data/attributes.npy"
attributes = np.load(ATTR_PATH)
NUM_CLASSES = attributes.shape[0]
NUM_ATTR = attributes.shape[1]

print("\nAttributes shape:", attributes.shape)
print("NUM_CLASSES:", NUM_CLASSES, "| NUM_ATTR:", NUM_ATTR)

Loading train/val dataset from: processed_bird_data
Train size: 3337
Val size: 589

Loading test dataset from: processed_bird_test_data
Test size: 4000

Attributes shape: (200, 312)
NUM_CLASSES: 200 | NUM_ATTR: 312


In [33]:
from torchvision.transforms import InterpolationMode

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=25, interpolation=InterpolationMode.BILINEAR),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

eval_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

In [34]:
class BirdTrainDataset(Dataset):
    def __init__(self, hf_dataset, attributes, transform=None):
        self.ds = hf_dataset
        self.attributes = attributes.astype("float32")
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        img = item["image"]
        if isinstance(img, Image.Image):
            img = img.convert("RGB")
        label = int(item["label"]) 

        if self.transform is not None:
            img = self.transform(img)

        attr_vec = self.attributes[label]
        attr_vec = torch.from_numpy(attr_vec)

        return img, attr_vec, label


class BirdTestDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.ds = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        img = item["image"]
        if isinstance(img, Image.Image):
            img = img.convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        img_id = int(item["id"])
        return img, img_id


train_dataset = BirdTrainDataset(train_hf, attributes, transform=train_transform)
val_dataset   = BirdTrainDataset(val_hf,   attributes, transform=eval_transform)
test_dataset  = BirdTestDataset(test_hf,   transform=eval_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [35]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=192):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = img_size//patch_size
        self.num_patches = self.grid_size**2

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        # x [B,3,H,W]
        x = self.proj(x) #[B,embed_dim,H/P,W/P]
        x = x.flatten(2) #[B,embed_dim,num_patches]
        x = x.transpose(1, 2) #[B,num_patches,embed_dim]
        return x


class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim=192, num_heads=3, mlp_ratio=4.0, drop=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=drop, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)

        hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(drop),
        )

    def forward(self, x):
        #x [B,N,D]
        x_norm = self.norm1(x)
        attn_out, _ = self.attn(x_norm, x_norm, x_norm)
        x = x + attn_out
        x = x + self.mlp(self.norm2(x))
        return x


class SimpleViTWithAttributes(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3,
                 num_classes=200, num_attr=312, embed_dim=192, depth=6,
                 num_heads=3, mlp_ratio=4.0, drop=0.1):
        super().__init__()

        self.patch_embed = PatchEmbed(img_size, patch_size, in_chans, embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(drop)

        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, mlp_ratio, drop)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)

        self.head_class = nn.Linear(embed_dim, num_classes)
        # głowa atrybutowa
        self.head_attr  = nn.Linear(embed_dim, num_attr)

        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.head_class.weight, std=0.02)
        nn.init.trunc_normal_(self.head_attr.weight, std=0.02)

    def forward(self, x):
        # x:[B,3,224,224]
        B = x.size(0)
        x = self.patch_embed(x) #[B,N,D]

        cls_tokens = self.cls_token.expand(B,-1,-1) #[B,1,D]
        x = torch.cat((cls_tokens, x), dim=1) #[B,1+N,D]
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        cls = x[:, 0] #[B,D]

        logits = self.head_class(cls)
        attr_pred = self.head_attr(cls)
        return logits, attr_pred
    
model = SimpleViTWithAttributes(
    img_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    num_classes=NUM_CLASSES,
    num_attr=NUM_ATTR,
    embed_dim=192,
    depth=6,
    num_heads=3,
    mlp_ratio=4.0,
    drop=0.1
).to(DEVICE)

model

SimpleViTWithAttributes(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0-5): 6 x TransformerEncoderBlock(
      (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=192, out_features=192, bias=True)
      )
      (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=192, out_features=768, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.1, inplace=False)
        (3): Linear(in_features=768, out_features=192, bias=True)
        (4): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  (head_class): Linear(in_features=192, out_features=200, bias=True)
  (head_attr): Linear(in_features=192, out_features=312, b

In [ ]:
criterion_class = nn.CrossEntropyLoss()
criterion_attr  = nn.MSELoss()
LAMBDA_ATTR = 0.05

In [37]:
def train_one_epoch(epoch_idx):
    model.train()
    total_loss = 0.0
    total_cls  = 0.0
    total_attr = 0.0
    correct = 0
    samples = 0

    for batch_idx, (imgs, attr_targets, labels) in enumerate(train_loader):
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        attr_targets = attr_targets.to(DEVICE)

        optimizer.zero_grad()

        logits, attr_pred = model(imgs)

        loss_cls = criterion_class(logits, labels)
        loss_attr = criterion_attr(attr_pred, attr_targets)
        loss = loss_cls + LAMBDA_ATTR * loss_attr

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        total_cls  += loss_cls.item() * imgs.size(0)
        total_attr += loss_attr.item() * imgs.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        samples += imgs.size(0)

        if batch_idx % 20 == 0:
            print(f"[Epoch {epoch_idx}] Batch {batch_idx}/{len(train_loader)} "
                  f"loss={loss.item():.4f}")

    avg_loss = total_loss / samples
    avg_cls  = total_cls  / samples
    avg_attr = total_attr / samples
    acc = correct / samples
    return avg_loss, avg_cls, avg_attr, acc


def evaluate():
    model.eval()
    total_loss = 0.0
    total_cls  = 0.0
    total_attr = 0.0
    correct = 0
    samples = 0

    with torch.no_grad():
        for imgs, attr_targets, labels in val_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            attr_targets = attr_targets.to(DEVICE)

            logits, attr_pred = model(imgs)

            loss_cls = criterion_class(logits, labels)
            loss_attr = criterion_attr(attr_pred, attr_targets)
            loss = loss_cls + LAMBDA_ATTR * loss_attr

            total_loss += loss.item() * imgs.size(0)
            total_cls  += loss_cls.item() * imgs.size(0)
            total_attr += loss_attr.item() * imgs.size(0)

            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            samples += imgs.size(0)

    avg_loss = total_loss / samples
    avg_cls  = total_cls  / samples
    avg_attr = total_attr / samples
    acc = correct / samples
    return avg_loss, avg_cls, avg_attr, acc

In [38]:
def reset_seed(seed=RANDOM_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def create_model_for_trial(drop: float):
    # rebuild your ViT with this trial's dropout
    model = SimpleViTWithAttributes(
        img_size=IMG_SIZE,
        patch_size=PATCH_SIZE,
        num_classes=NUM_CLASSES,
        num_attr=NUM_ATTR,
        embed_dim=192,   # fixed for now
        depth=6,
        num_heads=3,
        mlp_ratio=4.0,
        drop=drop
    ).to(DEVICE)
    return model


def objective(trial: optuna.Trial):
    global model, optimizer, scheduler, LAMBDA_ATTR

    # 1. Sample hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 5e-4, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    drop = trial.suggest_float("drop", 0.0, 0.3)
    lambda_attr = trial.suggest_float("lambda_attr", 0.01, 0.2, log=True)

    # 2. Reset random seeds for reproducibility
    reset_seed()

    # 3. Create model & optimizer for this trial
    model = create_model_for_trial(drop)

    LAMBDA_ATTR = lambda_attr

    optimizer = optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )

    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=TUNE_EPOCHS
    )

    best_val_acc = 0.0

    # 4. Train for a small number of epochs
    for epoch in range(1, TUNE_EPOCHS + 1):
        train_loss, train_cls, train_attr, train_acc = train_one_epoch(epoch)
        val_loss, val_cls, val_attr, val_acc = evaluate()

        scheduler.step()

        # Report to Optuna (for pruning)
        trial.report(val_acc, step=epoch)

        # Optional: prune bad trials early
        if trial.should_prune():
            raise optuna.TrialPruned()

        if val_acc > best_val_acc:
            best_val_acc = val_acc

    # We want to maximize validation accuracy
    return best_val_acc


In [39]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=2),
)

study.optimize(objective, n_trials=25)

print("Best value:", study.best_trial.value)
print("Best params:", study.best_trial.params)

[I 2025-12-10 18:20:42,674] A new study created in memory with name: no-name-9b0d2cc0-70d0-4698-8d9d-f880548e8f7e


[Epoch 1] Batch 0/105 loss=5.4260
[Epoch 1] Batch 20/105 loss=5.3527
[Epoch 1] Batch 40/105 loss=5.2618
[Epoch 1] Batch 60/105 loss=5.2006
[Epoch 1] Batch 80/105 loss=5.3158
[Epoch 1] Batch 100/105 loss=5.1076
[Epoch 2] Batch 0/105 loss=5.1222
[Epoch 2] Batch 20/105 loss=5.2400
[Epoch 2] Batch 40/105 loss=5.1746
[Epoch 2] Batch 60/105 loss=5.2009
[Epoch 2] Batch 80/105 loss=5.1619
[Epoch 2] Batch 100/105 loss=5.1624
[Epoch 3] Batch 0/105 loss=5.1895
[Epoch 3] Batch 20/105 loss=5.0314
[Epoch 3] Batch 40/105 loss=5.0652
[Epoch 3] Batch 60/105 loss=5.0949
[Epoch 3] Batch 80/105 loss=5.2163
[Epoch 3] Batch 100/105 loss=5.0958
[Epoch 4] Batch 0/105 loss=5.0459
[Epoch 4] Batch 20/105 loss=5.2918
[Epoch 4] Batch 40/105 loss=5.0723
[Epoch 4] Batch 60/105 loss=4.9639
[Epoch 4] Batch 80/105 loss=5.0111
[Epoch 4] Batch 100/105 loss=5.0958
[Epoch 5] Batch 0/105 loss=4.9436
[Epoch 5] Batch 20/105 loss=5.0018
[Epoch 5] Batch 40/105 loss=5.0140
[Epoch 5] Batch 60/105 loss=5.1614
[Epoch 5] Batch 80/10

[I 2025-12-10 19:02:49,787] Trial 0 finished with value: 0.013582342954159592 and parameters: {'lr': 4.3284502212938785e-05, 'weight_decay': 0.006351221010640699, 'drop': 0.21959818254342153, 'lambda_attr': 0.060099747183803134}. Best is trial 0 with value: 0.013582342954159592.


[Epoch 1] Batch 0/105 loss=5.4528
[Epoch 1] Batch 20/105 loss=5.3027
[Epoch 1] Batch 40/105 loss=5.2589
[Epoch 1] Batch 60/105 loss=5.2371
[Epoch 1] Batch 80/105 loss=5.3567
[Epoch 1] Batch 100/105 loss=5.1825
[Epoch 2] Batch 0/105 loss=5.1942
[Epoch 2] Batch 20/105 loss=5.2541
[Epoch 2] Batch 40/105 loss=5.2165
[Epoch 2] Batch 60/105 loss=5.2089
[Epoch 2] Batch 80/105 loss=5.1774
[Epoch 2] Batch 100/105 loss=5.2399
[Epoch 3] Batch 0/105 loss=5.1988
[Epoch 3] Batch 20/105 loss=5.0773
[Epoch 3] Batch 40/105 loss=5.1182
[Epoch 3] Batch 60/105 loss=5.1042
[Epoch 3] Batch 80/105 loss=5.2409
[Epoch 3] Batch 100/105 loss=5.0990
[Epoch 4] Batch 0/105 loss=5.0704
[Epoch 4] Batch 20/105 loss=5.3023
[Epoch 4] Batch 40/105 loss=5.0692
[Epoch 4] Batch 60/105 loss=5.0104
[Epoch 4] Batch 80/105 loss=5.0952
[Epoch 4] Batch 100/105 loss=5.1612
[Epoch 5] Batch 0/105 loss=4.9891
[Epoch 5] Batch 20/105 loss=5.0519
[Epoch 5] Batch 40/105 loss=5.0897
[Epoch 5] Batch 60/105 loss=5.2013
[Epoch 5] Batch 80/10

[I 2025-12-10 19:29:32,060] Trial 1 finished with value: 0.011884550084889643 and parameters: {'lr': 1.8410729205738674e-05, 'weight_decay': 4.207053950287936e-06, 'drop': 0.017425083650459836, 'lambda_attr': 0.13394334706750485}. Best is trial 0 with value: 0.013582342954159592.


...

[Epoch 1] Batch 0/105 loss=5.4457
[Epoch 1] Batch 20/105 loss=5.3135
[Epoch 1] Batch 40/105 loss=5.1845
[Epoch 1] Batch 60/105 loss=5.1059
[Epoch 1] Batch 80/105 loss=5.2724
[Epoch 1] Batch 100/105 loss=4.9158
[Epoch 2] Batch 0/105 loss=4.9597
[Epoch 2] Batch 20/105 loss=5.0515
[Epoch 2] Batch 40/105 loss=4.9289
[Epoch 2] Batch 60/105 loss=5.0932
[Epoch 2] Batch 80/105 loss=4.9823
[Epoch 2] Batch 100/105 loss=4.9983
[Epoch 3] Batch 0/105 loss=4.9885
[Epoch 3] Batch 20/105 loss=4.7443
[Epoch 3] Batch 40/105 loss=4.8461
[Epoch 3] Batch 60/105 loss=4.8727
[Epoch 3] Batch 80/105 loss=5.1413
[Epoch 3] Batch 100/105 loss=4.9635
[Epoch 4] Batch 0/105 loss=4.7503
[Epoch 4] Batch 20/105 loss=5.1217
[Epoch 4] Batch 40/105 loss=4.8487
[Epoch 4] Batch 60/105 loss=4.7717
[Epoch 4] Batch 80/105 loss=4.5665
[Epoch 4] Batch 100/105 loss=4.7145
[Epoch 5] Batch 0/105 loss=4.7232
[Epoch 5] Batch 20/105 loss=4.8542
[Epoch 5] Batch 40/105 loss=4.6776
[Epoch 5] Batch 60/105 loss=4.7830
[Epoch 5] Batch 80/10

[I 2025-12-11 06:43:18,154] Trial 23 finished with value: 0.022071307300509338 and parameters: {'lr': 0.00030777398303933327, 'weight_decay': 1.5558205341022404e-06, 'drop': 0.25716261354321895, 'lambda_attr': 0.03179408669133864}. Best is trial 11 with value: 0.023769100169779286.


[Epoch 1] Batch 0/105 loss=5.4390
[Epoch 1] Batch 20/105 loss=5.3236
[Epoch 1] Batch 40/105 loss=5.1836
[Epoch 1] Batch 60/105 loss=5.0769
[Epoch 1] Batch 80/105 loss=5.2655
[Epoch 1] Batch 100/105 loss=4.9200
[Epoch 2] Batch 0/105 loss=4.9859
[Epoch 2] Batch 20/105 loss=5.0699
[Epoch 2] Batch 40/105 loss=4.9367
[Epoch 2] Batch 60/105 loss=5.0837
[Epoch 2] Batch 80/105 loss=4.9637
[Epoch 2] Batch 100/105 loss=4.9914


[I 2025-12-11 06:57:08,023] Trial 24 pruned. 


Best value: 0.023769100169779286
Best params: {'lr': 0.0004273669312062059, 'weight_decay': 2.3739423955292538e-05, 'drop': 0.10912464381815776, 'lambda_attr': 0.010368427651183114}


{'lr': 4.3284502212938785e-05, 'weight_decay': 0.006351221010640699, 'drop': 0.21959818254342153, 'lambda_attr': 0.060099747183803134}

In [40]:
best_params = study.best_trial.params
best_lr           = best_params["lr"]
best_weight_decay = best_params["weight_decay"]
best_drop         = best_params["drop"]
best_lambda_attr  = best_params["lambda_attr"]

reset_seed()

model = SimpleViTWithAttributes(
    img_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    num_classes=NUM_CLASSES,
    num_attr=NUM_ATTR,
    embed_dim=192,
    depth=6,
    num_heads=3,
    mlp_ratio=4.0,
    drop=best_drop,
).to(DEVICE)

LAMBDA_ATTR = best_lambda_attr

optimizer = optim.AdamW(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

best_val_acc = 0.0

for epoch in range(1, EPOCHS + 1):
    print(f"\n[FINAL TRAIN] Epoch {epoch}/{EPOCHS}")
    train_one_epoch(epoch)
    _, _, _, val_acc = evaluate()
    scheduler.step()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "vit_best_model_optimized.pth")
        print("Saved new best optimized model")



[FINAL TRAIN] Epoch 1/25
[Epoch 1] Batch 0/105 loss=5.4301
[Epoch 1] Batch 20/105 loss=5.3295
[Epoch 1] Batch 40/105 loss=5.2124
[Epoch 1] Batch 60/105 loss=5.0611
[Epoch 1] Batch 80/105 loss=5.2928
[Epoch 1] Batch 100/105 loss=4.8667
Saved new best optimized model

[FINAL TRAIN] Epoch 2/25
[Epoch 2] Batch 0/105 loss=4.9661
[Epoch 2] Batch 20/105 loss=5.0323
[Epoch 2] Batch 40/105 loss=4.7495
[Epoch 2] Batch 60/105 loss=5.0976
[Epoch 2] Batch 80/105 loss=4.9452
[Epoch 2] Batch 100/105 loss=5.0395

[FINAL TRAIN] Epoch 3/25
[Epoch 3] Batch 0/105 loss=4.9850
[Epoch 3] Batch 20/105 loss=4.6469
[Epoch 3] Batch 40/105 loss=4.7907
[Epoch 3] Batch 60/105 loss=4.9544
[Epoch 3] Batch 80/105 loss=5.1524
[Epoch 3] Batch 100/105 loss=4.9507

[FINAL TRAIN] Epoch 4/25
[Epoch 4] Batch 0/105 loss=4.6722
[Epoch 4] Batch 20/105 loss=5.3330
[Epoch 4] Batch 40/105 loss=4.8650
[Epoch 4] Batch 60/105 loss=4.7634
[Epoch 4] Batch 80/105 loss=4.5779
[Epoch 4] Batch 100/105 loss=4.8708
Saved new best optimized 

In [41]:
print("Number of finished trials:", len(study.trials))
print("Best trial value (val_acc):", study.best_trial.value)
print("Best hyperparameters:")
for k, v in study.best_trial.params.items():
    print(f"  {k}: {v}")


Number of finished trials: 25
Best trial value (val_acc): 0.023769100169779286
Best hyperparameters:
  lr: 0.0004273669312062059
  weight_decay: 2.3739423955292538e-05
  drop: 0.10912464381815776
  lambda_attr: 0.010368427651183114


In [42]:
best_vit = SimpleViTWithAttributes(
    img_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    num_classes=NUM_CLASSES,
    num_attr=NUM_ATTR,
    embed_dim=192,
    depth=6,
    num_heads=3,
    mlp_ratio=4.0,
    drop=0.1
).to(DEVICE)

best_vit.load_state_dict(torch.load("vit_best_model.pth", map_location=DEVICE))
best_vit.eval()

all_ids = []
all_preds = []

with torch.no_grad():
    for imgs, img_ids in test_loader:
        imgs = imgs.to(DEVICE)
        logits, attr_pred = best_vit(imgs)
        preds = logits.argmax(dim=1).cpu().numpy()

        all_ids.extend(img_ids.numpy().tolist())
        all_preds.extend(preds.tolist())

kaggle_labels = [p + 1 for p in all_preds]

submission_vit = pd.DataFrame({
    "id": all_ids,
    "label": kaggle_labels
}).sort_values("id")

print(submission_vit.head())

submission_vit.to_csv("vit_submission.csv", index=False)
print("\nSaved vit_submission.csv")

   id  label
0   1     33
1   2     55
2   3     28
3   4     34
4   5     16

Saved vit_submission.csv
